## COSI123A Homework 5 Nicole Meng

Task: There are 10 data sets (5 in folder a and 5  in fold b) in the attached zip file (HW5data.zip). Each data set has a training subset (for example, ‘Train_1a.csv’) and a test subset (for example, ‘TestX_1a.csv’).  Build a regression model using the training subset and apply the model to the test subset. Each row in a data file is a sample. The training data files contain the desired outputs. The headers of the data files indicate the feature columns and the output columns. The test files do not contain the outputs. Save all your prediction results in a file named “hw5_result.csv” following the format provided in the template file “hw5_result_template.csv”.  The order of the predictions in the result file should be the same to the order of their samples in the test file.

Requirements: In addition to the result file “hw5_result.csv”, please submit your codes and a written report explaining your preprocessing method if any and your regression model. You can use any existing package/function, and should clearly specify your chosen settings and explain why your choices.

Grading: The score will be calculated in the following way. Report takes 30% and prediction performance takes another 70%.  For prediction part, first, the root-mean-square error (RMSE) is calculated for each submission. The RMSEs from all students will be ranked.  The best prediction (i.e., the lowest RMSE) will get 70.  The scores of the rest predictions will be linearly scaled between 66.5 and 35 as:  66.5 – (rank-1)*(66.5 – 35)/(number of students made submissions - 1).  You will receive 0 if you produce random predictions or do not submit your prediction results.

Tips: There may exist outliers or redundant variables in the data, you can use any kind of preprocessing method, prediction model and parameter settings you want, but linear regression model is recommended. You may want to use the same method/setting for datasets in folder a, and the same method/setting for datasets in folder b.

### Import Packages

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model

import math
import os
import random

### Read Training CSV Files

In [2]:
#Read in files in folder a
train_1a=pd.read_csv("HW5data/a/1/Train_1a.csv", encoding="utf-8") 
train_2a=pd.read_csv("HW5data/a/2/Train_2a.csv", encoding="utf-8") 
train_3a=pd.read_csv("HW5data/a/3/Train_3a.csv", encoding="utf-8") 
train_4a=pd.read_csv("HW5data/a/4/Train_4a.csv", encoding="utf-8") 
train_5a=pd.read_csv("HW5data/a/5/Train_5a.csv", encoding="utf-8") 

#Read in files in folder b
train_1b=pd.read_csv("HW5data/b/1/TrainX_1b.csv", encoding="utf-8") 
train_2b=pd.read_csv("HW5data/b/2/TrainX_2b.csv", encoding="utf-8") 
train_3b=pd.read_csv("HW5data/b/3/TrainX_3b.csv", encoding="utf-8") 
train_4b=pd.read_csv("HW5data/b/4/TrainX_4b.csv", encoding="utf-8") 
train_5b=pd.read_csv("HW5data/b/5/TrainX_5b.csv", encoding="utf-8") 

### Read Testing CSV Files

In [3]:
#Read in files in folder a
test_1a=pd.read_csv("HW5data/a/1/Test_1a.csv", encoding="utf-8") 
test_2a=pd.read_csv("HW5data/a/2/Test_2a.csv", encoding="utf-8") 
test_3a=pd.read_csv("HW5data/a/3/Test_3a.csv", encoding="utf-8") 
test_4a=pd.read_csv("HW5data/a/4/Test_4a.csv", encoding="utf-8") 
test_5a=pd.read_csv("HW5data/a/5/Test_5a.csv", encoding="utf-8") 

#Read in files in folder b
test_1b=pd.read_csv("HW5data/b/1/Test_1b.csv", encoding="utf-8") 
test_2b=pd.read_csv("HW5data/b/2/Test_2b.csv", encoding="utf-8") 
test_3b=pd.read_csv("HW5data/b/3/Test_3b.csv", encoding="utf-8") 
test_4b=pd.read_csv("HW5data/b/4/Test_4b.csv", encoding="utf-8") 
test_5b=pd.read_csv("HW5data/b/5/Test_5b.csv", encoding="utf-8") 

In [4]:
train_1a

,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Feature9,Feature10,Feature11,Feature12,Feature13,Output
0,0.96971,-0.21141,0.019567,-0.104990,0.49927,0.21940,-0.376100,-0.77941,-0.62888,0.111550,-0.063305,0.44952,0.92620,-0.980010
1,1.12910,1.72620,2.253300,1.971600,1.49180,0.83064,-0.301950,-0.35494,-0.15466,0.265290,-0.472900,-2.28370,-1.44680,0.000000
2,0.58824,0.20883,0.267020,0.792490,0.98504,0.85893,-0.035307,-0.72672,-0.65232,0.289640,-0.671170,-0.78219,-3.29500,0.000000
3,0.85289,0.97132,-0.209330,0.814060,0.67885,0.47691,-0.038935,0.22225,-0.67780,-0.590720,0.491200,1.16010,2.41400,0.313210
4,-1.90190,-1.20480,-1.103000,-0.050817,1.11800,0.61725,1.138100,1.83370,3.39220,4.211000,0.561280,0.50147,-1.33780,0.095141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,0.17290,0.51523,0.373760,-0.096055,-0.10243,0.26491,-0.567820,-1.04950,-0.78011,-0.572970,0.963600,1.78170,-0.89378,-0.615380
173,1.22030,2.29180,1.871800,1.693800,1.87870,1.17300,1.438900,1.53080,0.67961,-0.066166,-3.446600,-3.63690,-1.25180,0.246690
174,0.83611,0.83100,0.336980,0.261010,0.42000,0.30603,0.180150,-0.23744,-0.33007,-0.819680,-0.229150,1.09760,-0.31218,0.141440
175,-0.72359,-0.89052,-1.347900,-0.616180,-1.28950,-0.65520,-0.413530,-0.73398,0.19131,-0.103780,1.363300,-1.60910,1.58260,-0.509540


### Model for a1

In [5]:
train_X = StandardScaler().fit_transform(train_1a.iloc[:,:-1])
train_y = train_1a.iloc[:,-1]
model = linear_model.ARDRegression()
model.fit(train_X, train_y)
test_X = StandardScaler().fit_transform(test_1a)
test_y = model.predict(test_X)

results = pd.read_csv('hw5_result_template.csv')
results['1a'] = test_y

## Now we will do the same thing for the rest of the train and test data

### Model for a2

In [6]:
train_X = StandardScaler().fit_transform(train_2a.iloc[:,:-1])
train_y = train_2a.iloc[:,-1]
model = linear_model.ARDRegression()
model.fit(train_X, train_y)
test_X = StandardScaler().fit_transform(test_2a)
test_y = model.predict(test_X)

results['2a'] = test_y

### Model for a3

In [7]:
train_X = StandardScaler().fit_transform(train_3a.iloc[:,:-1])
train_y = train_3a.iloc[:,-1]
model = linear_model.ARDRegression()
model.fit(train_X, train_y)
test_X = StandardScaler().fit_transform(test_3a)
test_y = model.predict(test_X)

results['3a'] = test_y
results

,Unnamed: 0,1a,2a,3a,4a,5a,1b,2b,3b,4b,5b
0,1,-0.573598,-0.084370,0.130647,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,0.095590,-0.344236,-0.041707,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,-0.253979,-0.007979,0.312099,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,-0.314832,0.070805,0.084742,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,-0.134445,0.056422,-0.411853,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,0.127790,-0.217426,-0.044842,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,0.027100,-0.148462,0.012358,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,-0.058754,0.143778,0.171725,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,-0.249479,-0.327549,0.076509,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,-0.280579,-0.349968,0.252963,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Model for a4

In [8]:
train_X = StandardScaler().fit_transform(train_4a.iloc[:,:-1])
train_y = train_4a.iloc[:,-1]
model = linear_model.ARDRegression()
model.fit(train_X, train_y)
test_X = StandardScaler().fit_transform(test_4a)
test_y = model.predict(test_X)

results['4a'] = test_y
results

,Unnamed: 0,1a,2a,3a,4a,5a,1b,2b,3b,4b,5b
0,1,-0.573598,-0.084370,0.130647,0.253822,NaN,NaN,NaN,NaN,NaN,NaN
1,2,0.095590,-0.344236,-0.041707,0.120405,NaN,NaN,NaN,NaN,NaN,NaN
2,3,-0.253979,-0.007979,0.312099,0.168265,NaN,NaN,NaN,NaN,NaN,NaN
3,4,-0.314832,0.070805,0.084742,-0.166624,NaN,NaN,NaN,NaN,NaN,NaN
4,5,-0.134445,0.056422,-0.411853,-0.119303,NaN,NaN,NaN,NaN,NaN,NaN
5,6,0.127790,-0.217426,-0.044842,0.037598,NaN,NaN,NaN,NaN,NaN,NaN
6,7,0.027100,-0.148462,0.012358,0.174850,NaN,NaN,NaN,NaN,NaN,NaN
7,8,-0.058754,0.143778,0.171725,-0.004740,NaN,NaN,NaN,NaN,NaN,NaN
8,9,-0.249479,-0.327549,0.076509,0.339256,NaN,NaN,NaN,NaN,NaN,NaN
9,10,-0.280579,-0.349968,0.252963,0.111476,NaN,NaN,NaN,NaN,NaN,NaN


### Model for a5

In [9]:
train_X = StandardScaler().fit_transform(train_5a.iloc[:,:-1])
train_y = train_5a.iloc[:,-1]
model = linear_model.ARDRegression()
model.fit(train_X, train_y)
test_X = StandardScaler().fit_transform(test_5a)
test_y = model.predict(test_X)

results['5a'] = test_y
results

,Unnamed: 0,1a,2a,3a,4a,5a,1b,2b,3b,4b,5b
0,1,-0.573598,-0.084370,0.130647,0.253822,0.025533,NaN,NaN,NaN,NaN,NaN
1,2,0.095590,-0.344236,-0.041707,0.120405,0.076654,NaN,NaN,NaN,NaN,NaN
2,3,-0.253979,-0.007979,0.312099,0.168265,0.077714,NaN,NaN,NaN,NaN,NaN
3,4,-0.314832,0.070805,0.084742,-0.166624,0.052099,NaN,NaN,NaN,NaN,NaN
4,5,-0.134445,0.056422,-0.411853,-0.119303,-0.028599,NaN,NaN,NaN,NaN,NaN
5,6,0.127790,-0.217426,-0.044842,0.037598,0.009012,NaN,NaN,NaN,NaN,NaN
6,7,0.027100,-0.148462,0.012358,0.174850,0.035310,NaN,NaN,NaN,NaN,NaN
7,8,-0.058754,0.143778,0.171725,-0.004740,0.042918,NaN,NaN,NaN,NaN,NaN
8,9,-0.249479,-0.327549,0.076509,0.339256,0.069679,NaN,NaN,NaN,NaN,NaN
9,10,-0.280579,-0.349968,0.252963,0.111476,0.069321,NaN,NaN,NaN,NaN,NaN


### Model for 1b

In [10]:
train_X = StandardScaler().fit_transform(train_1b.iloc[:,:-1])
train_y = train_1b.iloc[:,-1]
model = linear_model.ARDRegression()
model.fit(train_X, train_y)
test_X = StandardScaler().fit_transform(test_1b)
test_y = model.predict(test_X)


results['1b'] = test_y
results

,Unnamed: 0,1a,2a,3a,4a,5a,1b,2b,3b,4b,5b
0,1,-0.573598,-0.084370,0.130647,0.253822,0.025533,-0.271550,NaN,NaN,NaN,NaN
1,2,0.095590,-0.344236,-0.041707,0.120405,0.076654,-0.053352,NaN,NaN,NaN,NaN
2,3,-0.253979,-0.007979,0.312099,0.168265,0.077714,-0.145302,NaN,NaN,NaN,NaN
3,4,-0.314832,0.070805,0.084742,-0.166624,0.052099,-0.202214,NaN,NaN,NaN,NaN
4,5,-0.134445,0.056422,-0.411853,-0.119303,-0.028599,0.244383,NaN,NaN,NaN,NaN
5,6,0.127790,-0.217426,-0.044842,0.037598,0.009012,0.150086,NaN,NaN,NaN,NaN
6,7,0.027100,-0.148462,0.012358,0.174850,0.035310,0.179147,NaN,NaN,NaN,NaN
7,8,-0.058754,0.143778,0.171725,-0.004740,0.042918,-0.326403,NaN,NaN,NaN,NaN
8,9,-0.249479,-0.327549,0.076509,0.339256,0.069679,-0.324778,NaN,NaN,NaN,NaN
9,10,-0.280579,-0.349968,0.252963,0.111476,0.069321,-0.225316,NaN,NaN,NaN,NaN


### Model for 2b

In [11]:
train_X = StandardScaler().fit_transform(train_2b.iloc[:,:-1])
train_y = train_2b.iloc[:,-1]
model = linear_model.ARDRegression()
model.fit(train_X, train_y)
test_X = StandardScaler().fit_transform(test_2b)
test_y = model.predict(test_X)

results['2b'] = test_y
results

,Unnamed: 0,1a,2a,3a,4a,5a,1b,2b,3b,4b,5b
0,1,-0.573598,-0.084370,0.130647,0.253822,0.025533,-0.271550,-0.281920,NaN,NaN,NaN
1,2,0.095590,-0.344236,-0.041707,0.120405,0.076654,-0.053352,-0.293983,NaN,NaN,NaN
2,3,-0.253979,-0.007979,0.312099,0.168265,0.077714,-0.145302,-0.096155,NaN,NaN,NaN
3,4,-0.314832,0.070805,0.084742,-0.166624,0.052099,-0.202214,0.269781,NaN,NaN,NaN
4,5,-0.134445,0.056422,-0.411853,-0.119303,-0.028599,0.244383,-0.122381,NaN,NaN,NaN
5,6,0.127790,-0.217426,-0.044842,0.037598,0.009012,0.150086,-0.095003,NaN,NaN,NaN
6,7,0.027100,-0.148462,0.012358,0.174850,0.035310,0.179147,-0.108869,NaN,NaN,NaN
7,8,-0.058754,0.143778,0.171725,-0.004740,0.042918,-0.326403,-0.302843,NaN,NaN,NaN
8,9,-0.249479,-0.327549,0.076509,0.339256,0.069679,-0.324778,-0.540290,NaN,NaN,NaN
9,10,-0.280579,-0.349968,0.252963,0.111476,0.069321,-0.225316,0.184817,NaN,NaN,NaN


### Model for 3b

In [12]:
train_X = StandardScaler().fit_transform(train_3b.iloc[:,:-1])
train_y = train_3b.iloc[:,-1]
model = linear_model.ARDRegression()
model.fit(train_X, train_y)
test_X = StandardScaler().fit_transform(test_3b)
test_y = model.predict(test_X)

results['3b'] = test_y
results

,Unnamed: 0,1a,2a,3a,4a,5a,1b,2b,3b,4b,5b
0,1,-0.573598,-0.084370,0.130647,0.253822,0.025533,-0.271550,-0.281920,0.171300,NaN,NaN
1,2,0.095590,-0.344236,-0.041707,0.120405,0.076654,-0.053352,-0.293983,-0.358555,NaN,NaN
2,3,-0.253979,-0.007979,0.312099,0.168265,0.077714,-0.145302,-0.096155,-0.134066,NaN,NaN
3,4,-0.314832,0.070805,0.084742,-0.166624,0.052099,-0.202214,0.269781,0.177554,NaN,NaN
4,5,-0.134445,0.056422,-0.411853,-0.119303,-0.028599,0.244383,-0.122381,0.594931,NaN,NaN
5,6,0.127790,-0.217426,-0.044842,0.037598,0.009012,0.150086,-0.095003,0.211337,NaN,NaN
6,7,0.027100,-0.148462,0.012358,0.174850,0.035310,0.179147,-0.108869,0.390413,NaN,NaN
7,8,-0.058754,0.143778,0.171725,-0.004740,0.042918,-0.326403,-0.302843,-0.313238,NaN,NaN
8,9,-0.249479,-0.327549,0.076509,0.339256,0.069679,-0.324778,-0.540290,0.071254,NaN,NaN
9,10,-0.280579,-0.349968,0.252963,0.111476,0.069321,-0.225316,0.184817,0.098498,NaN,NaN


### Model 4b

In [13]:
train_X = StandardScaler().fit_transform(train_4b.iloc[:,:-1])
train_y = train_4b.iloc[:,-1]
model = linear_model.ARDRegression()
model.fit(train_X, train_y)
test_X = StandardScaler().fit_transform(test_4b)
test_y = model.predict(test_X)

results['4b'] = test_y
results

,Unnamed: 0,1a,2a,3a,4a,5a,1b,2b,3b,4b,5b
0,1,-0.573598,-0.084370,0.130647,0.253822,0.025533,-0.271550,-0.281920,0.171300,-0.047156,NaN
1,2,0.095590,-0.344236,-0.041707,0.120405,0.076654,-0.053352,-0.293983,-0.358555,0.100138,NaN
2,3,-0.253979,-0.007979,0.312099,0.168265,0.077714,-0.145302,-0.096155,-0.134066,0.010254,NaN
3,4,-0.314832,0.070805,0.084742,-0.166624,0.052099,-0.202214,0.269781,0.177554,-0.004205,NaN
4,5,-0.134445,0.056422,-0.411853,-0.119303,-0.028599,0.244383,-0.122381,0.594931,-0.055403,NaN
5,6,0.127790,-0.217426,-0.044842,0.037598,0.009012,0.150086,-0.095003,0.211337,-0.150399,NaN
6,7,0.027100,-0.148462,0.012358,0.174850,0.035310,0.179147,-0.108869,0.390413,0.045139,NaN
7,8,-0.058754,0.143778,0.171725,-0.004740,0.042918,-0.326403,-0.302843,-0.313238,-0.069634,NaN
8,9,-0.249479,-0.327549,0.076509,0.339256,0.069679,-0.324778,-0.540290,0.071254,-0.070438,NaN
9,10,-0.280579,-0.349968,0.252963,0.111476,0.069321,-0.225316,0.184817,0.098498,-0.056813,NaN


### Model 5b

In [14]:
train_X = StandardScaler().fit_transform(train_5b.iloc[:,:-1])
train_y = train_5b.iloc[:,-1]
model = linear_model.ARDRegression()
model.fit(train_X, train_y)
test_X = StandardScaler().fit_transform(test_5b)
test_y = model.predict(test_X)

results['5b'] = test_y
results

,Unnamed: 0,1a,2a,3a,4a,5a,1b,2b,3b,4b,5b
0,1,-0.573598,-0.084370,0.130647,0.253822,0.025533,-0.271550,-0.281920,0.171300,-0.047156,-0.023739
1,2,0.095590,-0.344236,-0.041707,0.120405,0.076654,-0.053352,-0.293983,-0.358555,0.100138,-0.007230
2,3,-0.253979,-0.007979,0.312099,0.168265,0.077714,-0.145302,-0.096155,-0.134066,0.010254,0.091842
3,4,-0.314832,0.070805,0.084742,-0.166624,0.052099,-0.202214,0.269781,0.177554,-0.004205,0.050639
4,5,-0.134445,0.056422,-0.411853,-0.119303,-0.028599,0.244383,-0.122381,0.594931,-0.055403,0.075147
5,6,0.127790,-0.217426,-0.044842,0.037598,0.009012,0.150086,-0.095003,0.211337,-0.150399,0.029832
6,7,0.027100,-0.148462,0.012358,0.174850,0.035310,0.179147,-0.108869,0.390413,0.045139,0.001835
7,8,-0.058754,0.143778,0.171725,-0.004740,0.042918,-0.326403,-0.302843,-0.313238,-0.069634,-0.094425
8,9,-0.249479,-0.327549,0.076509,0.339256,0.069679,-0.324778,-0.540290,0.071254,-0.070438,0.031390
9,10,-0.280579,-0.349968,0.252963,0.111476,0.069321,-0.225316,0.184817,0.098498,-0.056813,-0.018392


In [15]:
results.to_csv('hw5_results.csv', index=False)